<a href="https://colab.research.google.com/github/RhettTitus/Hubitat-Apps/blob/master/NBA_Props_Ortg%2C_Drtg%2C_Pace_Matchups_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary package for NBA data
!pip install nba_api

import json
import pandas as pd
import requests
from datetime import datetime
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, teamgamelog

# Set display options for better readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Team name mapping: Full names to abbreviations
team_name_mapping = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "LA Clippers": "LAC",  # Corrected abbreviation
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS",
}

# Function to fetch today's NBA schedule
def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Set timezone to EST
        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({
                        'home_team': game['h']['ta'],
                        'away_team': game['v']['ta'],
                        'home_team_id': game['h']['tid'],
                        'away_team_id': game['v']['tid']
                    })

        if not games_today:
            print(f"No games scheduled for {today}.")
        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

# Fetch team metrics
def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(
            season="2024-25", season_type="Regular Season", league_id="00"
        )
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

# Analyze matchups based on metrics
def analyze_matchups(schedule, team_metrics):
    matchups = []

    for game in schedule:
        home_team = game['home_team']
        away_team = game['away_team']

        # Match team metrics by abbreviation
        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            print(f"Metrics not found for {home_team} or {away_team}")
            continue

        # Extract metrics
        home_ortg = home_metrics["E_OFF_RATING"].values[0]
        home_drtg = home_metrics["E_DEF_RATING"].values[0]
        home_pace = home_metrics["E_PACE"].values[0]

        away_ortg = away_metrics["E_OFF_RATING"].values[0]
        away_drtg = away_metrics["E_DEF_RATING"].values[0]
        away_pace = away_metrics["E_PACE"].values[0]

        # Calculate averages
        avg_ortg = (home_ortg + away_ortg) / 2
        avg_drtg = (home_drtg + away_drtg) / 2
        avg_pace = (home_pace + away_pace) / 2

        # Determine category
        if avg_ortg >= 115 and avg_pace >= 100:
            category = "High-scoring game (Favoring Points & Assists Props)" # High Ortg + High Pace
        elif avg_drtg < 110 and avg_pace <= 97:
            category = "Slow-paced game (Favoring Unders for Points Props)" # Low Drtg + Low Pace
        elif avg_ortg >= 115 and avg_drtg <= 110:
            category = "Fast-paced game (High Points Props)" # High Ortg + Low Drtg
        elif avg_pace >= 100:
            category = "Fast-paced game (More Possesions Big Upside for Points,Rebounds & Assists Props)"
        elif avg_drtg <110 and avg_pace <= 97:
            category = "Low-paced game (Likely to Hit Unders)"
        else:
            category = "Neutral Matchup"

        matchups.append({
            "Home Team": home_team,
            "Away Team": away_team,
            "Avg OffRtg": avg_ortg,
            "Avg DefRtg": avg_drtg,
            "Avg Pace": avg_pace,
            "Category": category
        })

    return pd.DataFrame(matchups)

# Main execution
schedule = fetch_schedule()
if schedule:
    print("Today's Schedule:")
    print(schedule)

    team_metrics = fetch_team_metrics()
    if not team_metrics.empty:
        print("\nTeam Metrics:")
        print(team_metrics.head(30))

        # Analyze matchups
        matchups_df = analyze_matchups(schedule, team_metrics)
        if not matchups_df.empty:
            print("\nMatchups Analysis:")
            print(matchups_df)
        else:
            print("No matchups could be analyzed.")
    else:
        print("Failed to fetch team metrics.")
else:
    print("Failed to fetch today's schedule.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 594.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.2/285.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 16.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'nba_api'

In [ ]:
!pip install nba_api

import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

def fetch_yesterdays_scores():
    est = pytz.timezone('US/Eastern')
    yesterday = (datetime.now(est) - timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        scoreboard = scoreboardv2.ScoreboardV2(game_date=yesterday)
        games = scoreboard.get_data_frames()[0]

        results = {}
        for _, game in games.iterrows():
            results[(game['HOME_TEAM_ABBREVIATION'], game['VISITOR_TEAM_ABBREVIATION'])] = game['PTS_HOME'] + game['PTS_VISITOR']

        if not results:
            raise ValueError("No scores found, fallback needed.")

        return results
    except Exception as e:
        print(f"Error fetching NBA API scores: {e}")
        return {}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def determine_best_evaluation_method(yesterday_scores):
    if not yesterday_scores:
        return "Absolute Thresholds"

    total_scores = list(yesterday_scores.values())
    median_score = pd.Series(total_scores).median()

    if median_score > 215:
        return "Percentiles"
    elif median_score < 200:
        return "X-score"
    else:
        return "Absolute Thresholds"

def generate_prop_message(avg_pace, avg_ortg, avg_drtg):
    if avg_pace > 100 and avg_ortg > 115:
        return "**High-scoring, fast-paced game – great for Points, Rebounds & Assists props.**"
    elif avg_pace < 98 and avg_drtg < 110:
        return "**Slow-paced, defensive matchup – consider Unders on Points props.**"
    elif avg_ortg > 116 and avg_drtg > 112:
        return "**Weak defenses + elite offense – Points and Assists props should hit.**"
    else:
        return "**Balanced matchup – props depend on player usage rates.**"

def categorize_matchup(avg_pace, avg_ortg, avg_drtg):
    if avg_pace > 100 and avg_ortg > 115:
        return "**Fast-paced, high-scoring game.**"
    elif avg_drtg < 109 and avg_pace < 98:
        return "**Defensive battle, low-scoring.**"
    elif avg_ortg > 116 and avg_drtg > 112:
        return "**Weak defenses, high upside for offense.**"
    return "**Balanced matchup.**"

def analyze_matchups(schedule, team_metrics):
    matchups = []

    for game in schedule:
        home_team, away_team = game['home_team'], game['away_team']

        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            continue

        home_ortg, home_drtg, home_pace = home_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        away_ortg, away_drtg, away_pace = away_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]

        avg_ortg = (home_ortg + away_ortg) / 2
        avg_drtg = (home_drtg + away_drtg) / 2
        avg_pace = (home_pace + away_pace) / 2

        category = categorize_matchup(avg_pace, avg_ortg, avg_drtg)
        prop_message = generate_prop_message(avg_pace, avg_ortg, avg_drtg)
        better_team = home_team if home_ortg > away_ortg else away_team

        matchups.append({
            "Home Team": home_team, "Away Team": away_team,
            "Category": category, "Prop Message": prop_message,
            "Better Team": better_team,
            "Home Stats": f"**OffRtg:** {home_ortg} | **DefRtg:** {home_drtg} | **Pace:** {home_pace}",
            "Away Stats": f"**OffRtg:** {away_ortg} | **DefRtg:** {away_drtg} | **Pace:** {away_pace}"
        })

    return matchups

yesterday_results = fetch_yesterdays_scores()
evaluation_method = determine_best_evaluation_method(yesterday_results)

schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

if schedule and not team_metrics.empty:
    matchups_analysis = analyze_matchups(schedule, team_metrics)

    for matchup in matchups_analysis:
        print(f"\n{matchup['Home Team']} vs {matchup['Away Team']}")
        print(f"  - **{matchup['Category']}**")
        print(f"  - {matchup['Prop Message']}")
        print(f"  - {matchup['Home Team']}: {matchup['Home Stats']}")
        print(f"  - {matchup['Away Team']}: {matchup['Away Stats']}")
        print(f"  - **Advantage:** {matchup['Better Team']}")


In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

def fetch_yesterdays_scores():
    est = pytz.timezone('US/Eastern')
    yesterday = (datetime.now(est) - timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        scoreboard = scoreboardv2.ScoreboardV2(game_date=yesterday)
        games = scoreboard.get_data_frames()[1]  # The second dataframe contains scores

        results = {}
        for _, game in games.iterrows():
            home_team = game['TEAM_ABBREVIATION'] if 'TEAM_ABBREVIATION' in game else team_name_mapping.get(game['TEAM_CITY_NAME'], None)
            if not home_team:
                continue  # Skip if team abbreviation isn't found

            away_team = None
            for _, other_game in games.iterrows():
                if other_game['GAME_ID'] == game['GAME_ID'] and other_game['TEAM_ID'] != game['TEAM_ID']:
                    away_team = other_game['TEAM_ABBREVIATION']
                    break

            if home_team and away_team:
                results[(home_team, away_team)] = game['PTS'] + other_game['PTS']

        if not results:
            raise ValueError("No scores found, fallback needed.")

        return results
    except Exception as e:
        print(f"❌ Error fetching NBA API scores: {e}")
        return {}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        return games_today
    except Exception as e:
        print(f"❌ Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"❌ Error fetching team metrics: {e}")
        return pd.DataFrame()

def determine_best_evaluation_method(yesterday_scores):
    if not yesterday_scores:
        return "Absolute Thresholds"

    total_scores = list(yesterday_scores.values())
    median_score = pd.Series(total_scores).median()

    if median_score > 215:
        return "Percentiles"
    elif median_score < 200:
        return "X-score"
    else:
        return "Absolute Thresholds"

def generate_prop_message(avg_pace, avg_ortg, avg_drtg):
    if avg_pace > 100 and avg_ortg > 115:
        return "**High-scoring, fast-paced game – great for Points, Rebounds & Assists props.**"
    elif avg_pace < 98 and avg_drtg < 110:
        return "**Slow-paced, defensive matchup – consider Unders on Points props.**"
    elif avg_ortg > 116 and avg_drtg > 112:
        return "**Weak defenses + elite offense – Points and Assists props should hit.**"
    else:
        return "**Balanced matchup – props depend on player usage rates.**"

def categorize_matchup(avg_pace, avg_ortg, avg_drtg):
    if avg_pace > 100 and avg_ortg > 115:
        return "**Fast-paced, high-scoring game.**"
    elif avg_drtg < 109 and avg_pace < 98:
        return "**Defensive battle, low-scoring.**"
    elif avg_ortg > 116 and avg_drtg > 112:
        return "**Weak defenses, high upside for offense.**"
    return "**Balanced matchup.**"

def analyze_matchups(schedule, team_metrics):
    matchups = []

    for game in schedule:
        home_team, away_team = game['home_team'], game['away_team']

        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            continue

        home_ortg, home_drtg, home_pace = home_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        away_ortg, away_drtg, away_pace = away_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]

        avg_ortg = (home_ortg + away_ortg) / 2
        avg_drtg = (home_drtg + away_drtg) / 2
        avg_pace = (home_pace + away_pace) / 2

        category = categorize_matchup(avg_pace, avg_ortg, avg_drtg)
        prop_message = generate_prop_message(avg_pace, avg_ortg, avg_drtg)
        better_team = home_team if home_ortg > away_ortg else away_team

        matchups.append({
            "Home Team": home_team, "Away Team": away_team,
            "Category": category, "Prop Message": prop_message,
            "Better Team": better_team,
            "Home Stats": f"**OffRtg:** {home_ortg} | **DefRtg:** {home_drtg} | **Pace:** {home_pace}",
            "Away Stats": f"**OffRtg:** {away_ortg} | **DefRtg:** {away_drtg} | **Pace:** {away_pace}"
        })

    return matchups

yesterday_results = fetch_yesterdays_scores()
evaluation_method = determine_best_evaluation_method(yesterday_results)

schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

if schedule and not team_metrics.empty:
    matchups_analysis = analyze_matchups(schedule, team_metrics)

    for matchup in matchups_analysis:
        print(f"\n{matchup['Home Team']} vs {matchup['Away Team']}")
        print(f"  - **{matchup['Category']}**")
        print(f"  - {matchup['Prop Message']}")
        print(f"  - {matchup['Home Stats']}")
        print(f"  - {matchup['Away Stats']}")
        print(f"  - **Advantage:** {matchup['Better Team']}")



CHA vs WAS
  - ****Balanced matchup.****
  - **Balanced matchup – props depend on player usage rates.**
  - **OffRtg:** 105.3 | **DefRtg:** 110.8 | **Pace:** 100.5
  - **OffRtg:** 102.0 | **DefRtg:** 116.9 | **Pace:** 104.3
  - **Advantage:** CHA

DET vs ATL
  - ****Balanced matchup.****
  - **Balanced matchup – props depend on player usage rates.**
  - **OffRtg:** 109.3 | **DefRtg:** 110.8 | **Pace:** 101.7
  - **OffRtg:** 108.5 | **DefRtg:** 111.3 | **Pace:** 105.9
  - **Advantage:** DET

NYK vs HOU
  - ****Balanced matchup.****
  - **Balanced matchup – props depend on player usage rates.**
  - **OffRtg:** 117.6 | **DefRtg:** 111.5 | **Pace:** 99.5
  - **OffRtg:** 110.6 | **DefRtg:** 106.3 | **Pace:** 101.7
  - **Advantage:** NYK

MEM vs SAS
  - ****Balanced matchup.****
  - **Balanced matchup – props depend on player usage rates.**
  - **OffRtg:** 114.8 | **DefRtg:** 107.9 | **Pace:** 107.3
  - **OffRtg:** 110.6 | **DefRtg:** 111.4 | **Pace:** 101.5
  - **Advantage:** MEM

MIN vs S

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

def fetch_yesterdays_scores():
    est = pytz.timezone('US/Eastern')
    yesterday = (datetime.now(est) - timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        scoreboard = scoreboardv2.ScoreboardV2(game_date=yesterday)
        games = scoreboard.get_data_frames()[0]

        results = {}
        for _, game in games.iterrows():
            results[(game['HOME_TEAM_ABBREVIATION'], game['VISITOR_TEAM_ABBREVIATION'])] = game['PTS_HOME'] + game['PTS_VISITOR']

        if not results:
            raise ValueError("No scores found, fallback needed.")

        return results
    except Exception as e:
        print(f"Error fetching NBA API scores: {e}")
        return {}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def categorize_matchup(home_ortg, home_drtg, home_pace, away_ortg, away_drtg, away_pace):
    avg_pace = (home_pace + away_pace) / 2
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2

    if avg_pace > 100 and avg_ortg > 115:
        return "**Fast-paced, high-scoring game.**"
    elif avg_drtg < 109 and avg_pace < 98:
        return "**Defensive battle, low-scoring.**"
    elif avg_ortg > 116 and avg_drtg > 112:
        return "**Weak defenses, high upside for offense.**"
    return "**Slight edge to one team based on offensive & defensive metrics.**"

def analyze_matchups(schedule, team_metrics):
    matchups = []

    for game in schedule:
        home_team, away_team = game['home_team'], game['away_team']

        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            continue

        home_ortg, home_drtg, home_pace = home_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        away_ortg, away_drtg, away_pace = away_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]

        category = categorize_matchup(home_ortg, home_drtg, home_pace, away_ortg, away_drtg, away_pace)

        if home_ortg > away_ortg and home_drtg < away_drtg:
            better_team = home_team
            advantage_reason = f"{home_team} has a stronger offense ({home_ortg} OffRtg) and better defense ({home_drtg} DefRtg) than {away_team}."
        elif away_ortg > home_ortg and away_drtg < home_drtg:
            better_team = away_team
            advantage_reason = f"{away_team} has a stronger offense ({away_ortg} OffRtg) and better defense ({away_drtg} DefRtg) than {home_team}."
        elif home_pace > away_pace:
            better_team = home_team
            advantage_reason = f"{home_team} plays at a faster pace ({home_pace} Pace), which could wear down {away_team}."
        else:
            better_team = away_team
            advantage_reason = f"{away_team} plays at a faster pace ({away_pace} Pace), potentially outpacing {home_team}."

        matchups.append({
            "Home Team": home_team, "Away Team": away_team,
            "Category": category, "Advantage Reason": advantage_reason,
            "Better Team": better_team,
            "Home Stats": f"**OffRtg:** {home_ortg} | **DefRtg:** {home_drtg} | **Pace:** {home_pace}",
            "Away Stats": f"**OffRtg:** {away_ortg} | **DefRtg:** {away_drtg} | **Pace:** {away_pace}"
        })

    return matchups

schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

if schedule and not team_metrics.empty:
    matchups_analysis = analyze_matchups(schedule, team_metrics)

    for matchup in matchups_analysis:
        print(f"\n{matchup['Home Team']} vs {matchup['Away Team']}")
        print(f"  - **{matchup['Category']}**")
        print(f"  - **Advantage:** {matchup['Better Team']} ({matchup['Advantage Reason']})")
        print(f"  - {matchup['Home Team']}: {matchup['Home Stats']}")
        print(f"  - {matchup['Away Team']}: {matchup['Away Stats']}")



CHA vs WAS
  - ****Slight edge to one team based on offensive & defensive metrics.****
  - **Advantage:** CHA (CHA has a stronger offense (105.3 OffRtg) and better defense (110.8 DefRtg) than WAS.)
  - CHA: **OffRtg:** 105.3 | **DefRtg:** 110.8 | **Pace:** 100.5
  - WAS: **OffRtg:** 102.0 | **DefRtg:** 116.9 | **Pace:** 104.3

DET vs ATL
  - ****Slight edge to one team based on offensive & defensive metrics.****
  - **Advantage:** DET (DET has a stronger offense (109.3 OffRtg) and better defense (110.8 DefRtg) than ATL.)
  - DET: **OffRtg:** 109.3 | **DefRtg:** 110.8 | **Pace:** 101.7
  - ATL: **OffRtg:** 108.5 | **DefRtg:** 111.3 | **Pace:** 105.9

NYK vs HOU
  - ****Slight edge to one team based on offensive & defensive metrics.****
  - **Advantage:** HOU (HOU plays at a faster pace (101.7 Pace), potentially outpacing NYK.)
  - NYK: **OffRtg:** 117.6 | **DefRtg:** 111.5 | **Pace:** 99.5
  - HOU: **OffRtg:** 110.6 | **DefRtg:** 106.3 | **Pace:** 101.7

MEM vs SAS
  - ****Slight edge 

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

def fetch_yesterdays_scores():
    est = pytz.timezone('US/Eastern')
    yesterday = (datetime.now(est) - timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        scoreboard = scoreboardv2.ScoreboardV2(game_date=yesterday)
        games = scoreboard.get_data_frames()[0]

        results = {}
        for _, game in games.iterrows():
            results[(game['HOME_TEAM_ABBREVIATION'], game['VISITOR_TEAM_ABBREVIATION'])] = game['PTS_HOME'] + game['PTS_VISITOR']

        if not results:
            raise ValueError("No scores found, fallback needed.")

        return results
    except Exception as e:
        print(f"Error fetching NBA API scores: {e}")
        return {}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def determine_best_evaluation_method(yesterday_scores):
    if not yesterday_scores:
        return "Absolute Thresholds"

    total_scores = list(yesterday_scores.values())
    median_score = pd.Series(total_scores).median()

    if median_score > 215:
        return "Percentiles"
    elif median_score < 200:
        return "X-score"
    else:
        return "Absolute Thresholds"

def generate_prop_message(avg_pace, avg_ortg, avg_drtg):
    props = []

    if avg_pace > 100 and avg_ortg > 115:
        props.append("Points Over")
        props.append("Assists Over")
    if avg_pace < 98 and avg_drtg < 110:
        props.append("Points Under")
        props.append("Rebounds Over")
    if avg_ortg > 116 and avg_drtg > 112:
        props.append("Points Over")
        props.append("Assists Over")
    if avg_drtg > 112 and avg_pace < 98:
        props.append("Rebounds Over")

    return ", ".join(props) if props else "Balanced matchup, depends on usage rates."

def categorize_matchup(avg_pace, avg_ortg, avg_drtg):
    if avg_pace > 100 and avg_ortg > 115:
        return "Fast-paced, high-scoring game."
    elif avg_drtg < 109 and avg_pace < 98:
        return "Defensive battle, low-scoring."
    elif avg_ortg > 116 and avg_drtg > 112:
        return "Weak defenses, high upside for offense."
    return "Balanced matchup."

def analyze_matchups(schedule, team_metrics):
    matchups = []

    for game in schedule:
        home_team, away_team = game['home_team'], game['away_team']

        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            continue

        home_ortg, home_drtg, home_pace = home_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        away_ortg, away_drtg, away_pace = away_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]

        avg_ortg = (home_ortg + away_ortg) / 2
        avg_drtg = (home_drtg + away_drtg) / 2
        avg_pace = (home_pace + away_pace) / 2

        category = categorize_matchup(avg_pace, avg_ortg, avg_drtg)
        prop_message = generate_prop_message(avg_pace, avg_ortg, avg_drtg)
        better_team = home_team if home_ortg > away_ortg else away_team

        advantage_reason = f"{better_team} has the edge because:\n"
        if home_ortg > away_ortg:
            advantage_reason += f"- **Higher Offensive Rating** ({home_ortg:.1f} vs {away_ortg:.1f})\n"
        if home_drtg < away_drtg:
            advantage_reason += f"- **Stronger Defense** ({home_drtg:.1f} vs {away_drtg:.1f})\n"
        if home_pace > away_pace:
            advantage_reason += f"- **Faster Pace** ({home_pace:.1f} vs {away_pace:.1f})\n"

        matchups.append({
            "Home Team": home_team, "Away Team": away_team,
            "Category": category, "Prop Message": prop_message,
            "Better Team": better_team,
            "Advantage Explanation": advantage_reason,
            "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace}",
            "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace}"
        })

    return matchups


In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        print(f"Today's Schedule: {games_today}")  # Debugging Line
        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)

        print(f"Fetched Team Metrics:\n{team_metrics}")  # Debugging Line
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def analyze_matchups(schedule, team_metrics):
    if not schedule:
        print("No games scheduled today.")  # Debugging
        return []

    if team_metrics.empty:
        print("Team metrics data is empty.")  # Debugging
        return []

    matchups = []

    for game in schedule:
        home_team, away_team = game['home_team'], game['away_team']

        home_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team]
        away_metrics = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team]

        if home_metrics.empty or away_metrics.empty:
            print(f"Skipping game {home_team} vs {away_team} (missing data)")  # Debugging
            continue

        home_ortg, home_drtg, home_pace = home_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        away_ortg, away_drtg, away_pace = away_metrics.iloc[0][["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]

        print(f"Processing {home_team} vs {away_team}")  # Debugging

        avg_ortg = (home_ortg + away_ortg) / 2
        avg_drtg = (home_drtg + away_drtg) / 2
        avg_pace = (home_pace + away_pace) / 2

        advantage_team = home_team if home_ortg > away_ortg else away_team
        advantage_reason = f"{advantage_team} has the edge due to:\n"
        if home_ortg > away_ortg:
            advantage_reason += f"- **Higher Offensive Rating** ({home_ortg:.1f} vs {away_ortg:.1f})\n"
        if home_drtg < away_drtg:
            advantage_reason += f"- **Better Defense** ({home_drtg:.1f} vs {away_drtg:.1f})\n"
        if home_pace > away_pace:
            advantage_reason += f"- **Faster Pace** ({home_pace:.1f} vs {away_pace:.1f})\n"

        prop_message = []
        if avg_pace > 100 and avg_ortg > 115:
            prop_message.append("Points Over")
            prop_message.append("Assists Over")
        if avg_pace < 98 and avg_drtg < 110:
            prop_message.append("Points Under")
            prop_message.append("Rebounds Over")
        if avg_ortg > 116 and avg_drtg > 112:
            prop_message.append("Points Over")
            prop_message.append("Assists Over")
        if avg_drtg > 112 and avg_pace < 98:
            prop_message.append("Rebounds Over")

        prop_message = ", ".join(prop_message) if prop_message else "Balanced matchup, depends on usage rates."

        matchups.append({
            "Home Team": home_team, "Away Team": away_team,
            "Better Team": advantage_team,
            "Advantage Explanation": advantage_reason,
            "Prop Bets": prop_message,
            "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace}",
            "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace}"
        })

    return matchups

# Run the Analysis
schedule = fetch_schedule()
team_metrics = fetch_team_metrics()
matchups = analyze_matchups(schedule, team_metrics)

# Print Results
for match in matchups:
    print(json.dumps(match, indent=4))


Today's Schedule: [{'home_team': 'CHA', 'away_team': 'WAS'}, {'home_team': 'DET', 'away_team': 'ATL'}, {'home_team': 'NYK', 'away_team': 'HOU'}, {'home_team': 'MEM', 'away_team': 'SAS'}, {'home_team': 'MIN', 'away_team': 'SAC'}, {'home_team': 'OKC', 'away_team': 'MIL'}, {'home_team': 'DEN', 'away_team': 'NOP'}, {'home_team': 'UTA', 'away_team': 'IND'}, {'home_team': 'GSW', 'away_team': 'ORL'}, {'home_team': 'POR', 'away_team': 'PHX'}]
Fetched Team Metrics:
                 TEAM_NAME  E_OFF_RATING  E_DEF_RATING  E_PACE  \
0        Memphis Grizzlies         114.8         107.9   107.3   
1            Atlanta Hawks         108.5         111.3   105.9   
2            Chicago Bulls         110.6         112.8   105.6   
3       Washington Wizards         102.0         116.9   104.3   
4           Denver Nuggets         116.5         111.8   102.9   
5    Oklahoma City Thunder         114.4         102.2   102.4   
6      Cleveland Cavaliers         120.4         108.7   102.3   
7     New O

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

# Create a reverse mapping from abbreviations to full names
team_abbreviation_to_name = {abbr: full_name for full_name, abbr in team_name_mapping.items()}

def fetch_yesterdays_scores():
    est = pytz.timezone('US/Eastern')
    yesterday = (datetime.now(est) - timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        scoreboard = scoreboardv2.ScoreboardV2(game_date=yesterday)
        games = scoreboard.get_data_frames()[0]

        results = {}
        for _, game in games.iterrows():
            results[(game['HOME_TEAM_ABBREVIATION'], game['VISITOR_TEAM_ABBREVIATION'])] = game['PTS_HOME'] + game['PTS_VISITOR']

        if not results:
            raise ValueError("No scores found, fallback needed.")

        return results
    except Exception as e:
        print(f"Error fetching NBA API scores: {e}")
        return {}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        print(f"Today's Schedule: {games_today}")  # Debugging Line
        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def determine_best_evaluation_method(yesterday_scores):
    if not yesterday_scores:
        return "Absolute Thresholds"

    total_scores = list(yesterday_scores.values())
    median_score = pd.Series(total_scores).median()

    if median_score > 215:
        return "Percentiles"
    elif median_score < 200:
        return "X-score"
    else:
        return "Absolute Thresholds"

def analyze_matchup(home_team, away_team, team_metrics):
    home_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team].iloc[0]
    away_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team].iloc[0]

    # Convert abbreviations to full names
    home_team_full = team_abbreviation_to_name.get(home_team, home_team)
    away_team_full = team_abbreviation_to_name.get(away_team, away_team)

    # Extract team metrics
    home_ortg, home_drtg, home_pace = home_stats["E_OFF_RATING"], home_stats["E_DEF_RATING"], home_stats["E_PACE"]
    away_ortg, away_drtg, away_pace = away_stats["E_OFF_RATING"], away_stats["E_DEF_RATING"], away_stats["E_PACE"]

    # Calculate Averages
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2
    avg_pace = (home_pace + away_pace) / 2

    # Determine category for prop bets
    if avg_ortg >= 115 and avg_pace >= 100:
        prop_bets = "High-scoring game (Favoring Points & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97:
        prop_bets = "Slow-paced game (Favoring Unders for Points Props)"
    elif avg_ortg >= 115 and avg_drtg <= 110:
        prop_bets = "Fast-paced game (High Points Props)"
    elif avg_pace >= 100:
        prop_bets = "Fast-paced game (More Possessions Big Upside for Points, Rebounds & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97:
        prop_bets = "Low-paced game (Likely to Hit Unders)"
    else:
        prop_bets = "Neutral Matchup"

    # Determine better team
    if home_ortg > away_ortg and home_drtg < away_drtg:
        better_team = home_team_full
        advantage_explanation = f"{home_team_full} has the edge due to:\n- **Higher Offensive Rating** ({home_ortg} vs {away_ortg})\n- **Better Defense** ({home_drtg} vs {away_drtg})"
    elif away_ortg > home_ortg and away_drtg < home_drtg:
        better_team = away_team_full
        advantage_explanation = f"{away_team_full} has the edge due to:\n- **Higher Offensive Rating** ({away_ortg} vs {home_ortg})\n- **Better Defense** ({away_drtg} vs {home_drtg})"
    else:
        better_team = "Balanced matchup"
        advantage_explanation = "Both teams are evenly matched in offense and defense."

    # Return the structured output
    return {
        "Home Team": home_team_full,
        "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace}",
        "Away Team": away_team_full,
        "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace}",
        "Better Team": better_team,
        "Advantage Explanation": advantage_explanation,
        "Prop Bets": prop_bets
    }

    return matchups

# Run the Analysis
schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

matchups = []  # Store matchup results

for game in schedule:
    home_team = game["home_team"]
    away_team = game["away_team"]

    # Ensure both teams exist in the metrics dataset before analyzing
    if home_team in team_metrics["TEAM_ABBREVIATION"].values and away_team in team_metrics["TEAM_ABBREVIATION"].values:
        matchup_result = analyze_matchup(home_team, away_team, team_metrics)
        matchups.append(matchup_result)
    else:
        print(f"Skipping matchup {home_team} vs {away_team} due to missing team metrics.")

# Print Results
for match in matchups:
    print(json.dumps(match, indent=4))

Today's Schedule: [{'home_team': 'PHI', 'away_team': 'DAL'}, {'home_team': 'CLE', 'away_team': 'BOS'}, {'home_team': 'BKN', 'away_team': 'HOU'}, {'home_team': 'TOR', 'away_team': 'NYK'}, {'home_team': 'CHI', 'away_team': 'MIA'}, {'home_team': 'LAC', 'away_team': 'LAL'}, {'home_team': 'POR', 'away_team': 'IND'}]
{
    "Home Team": "Philadelphia 76ers",
    "Home Stats": "OffRtg: 109.3 | DefRtg: 112.7 | Pace: 99.0",
    "Away Team": "Dallas Mavericks",
    "Away Stats": "OffRtg: 113.1 | DefRtg: 110.9 | Pace: 101.8",
    "Better Team": "Dallas Mavericks",
    "Advantage Explanation": "Dallas Mavericks has the edge due to:\n- **Higher Offensive Rating** (113.1 vs 109.3)\n- **Better Defense** (110.9 vs 112.7)",
    "Prop Bets": "Fast-paced game (More Possessions Big Upside for Points, Rebounds & Assists Props)"
}
{
    "Home Team": "Cleveland Cavaliers",
    "Home Stats": "OffRtg: 120.4 | DefRtg: 108.7 | Pace: 102.3",
    "Away Team": "Boston Celtics",
    "Away Stats": "OffRtg: 116.5 | Def

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics, scoreboardv2, teamgamelog

pd.set_option('display.max_columns', None)

team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

# Reverse mapping to get full team names
reverse_team_name_mapping = {v: k for k, v in team_name_mapping.items()}

def fetch_last_5_games(team_abbreviation):
    """Fetch last 5 games for a team and return their average points scored and allowed."""
    try:
        game_log = teamgamelog.TeamGameLog(team_id=team_abbreviation, season="2024-25", season_type="Regular Season")
        df = game_log.get_data_frames()[0]

        if df.empty:
            return {"Avg Points Scored": 0, "Avg Points Allowed": 0}

        # Get the last 5 games
        last_5_games = df.head(5)

        # Calculate average points scored and allowed
        avg_points_scored = np.mean(last_5_games["PTS"])
        avg_points_allowed = np.mean(last_5_games["PTS_OPP"])

        return {"Avg Points Scored": avg_points_scored, "Avg Points Allowed": avg_points_allowed}
    except Exception as e:
        print(f"Error fetching last 5 games for {team_abbreviation}: {e}")
        return {"Avg Points Scored": 0, "Avg Points Allowed": 0}

def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        print(f"Today's Schedule: {games_today}")  # Debugging Line
        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

def fetch_team_metrics():
    try:
        team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
        team_metrics = team_metrics_data.get_data_frames()[0]
        team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
        team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)
        return team_metrics
    except Exception as e:
        print(f"Error fetching team metrics: {e}")
        return pd.DataFrame()

def determine_best_evaluation_method(yesterday_scores):
    if not yesterday_scores:
        return "Absolute Thresholds"

    total_scores = list(yesterday_scores.values())
    median_score = pd.Series(total_scores).median()

    if median_score > 215:
        return "Percentiles"
    elif median_score < 200:
        return "X-score"
    else:
        return "Absolute Thresholds"

def analyze_matchup(home_team, away_team, team_metrics):
    """Analyze matchup based on offensive, defensive ratings, pace, and recent scoring trends."""
    home_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team].iloc[0]
    away_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team].iloc[0]

    # Fetch last 5-game averages
    home_recent = fetch_last_5_games(home_team)
    away_recent = fetch_last_5_games(away_team)

    # Extract season-long team metrics
    home_ortg, home_drtg, home_pace = home_stats["E_OFF_RATING"], home_stats["E_DEF_RATING"], home_stats["E_PACE"]
    away_ortg, away_drtg, away_pace = away_stats["E_OFF_RATING"], away_stats["E_DEF_RATING"], away_stats["E_PACE"]

    # Calculate overall averages
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2
    avg_pace = (home_pace + away_pace) / 2

    # Factor in recent scoring trends
    avg_recent_points = (home_recent["Avg Points Scored"] + away_recent["Avg Points Scored"]) / 2
    avg_recent_allowed = (home_recent["Avg Points Allowed"] + away_recent["Avg Points Allowed"]) / 2

    # Determine category for prop bets (using both season-long + recent form)
    if avg_ortg >= 115 and avg_pace >= 100 and avg_recent_points > 115:
        prop_bets = "High-scoring game (Favoring Points & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97 and avg_recent_allowed < 105:
        prop_bets = "Slow-paced game (Favoring Unders for Points Props)"
    elif avg_ortg >= 115 and avg_drtg <= 110 and avg_recent_points > 110:
        prop_bets = "Fast-paced game (High Points Props)"
    elif avg_pace >= 100 and avg_recent_points > 112:
        prop_bets = "Fast-paced game (More Possessions Big Upside for Points, Rebounds & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97 and avg_recent_allowed < 108:
        prop_bets = "Low-paced game (Likely to Hit Unders)"
    else:
        prop_bets = "Neutral Matchup"

    # Determine better team based on season-long and recent form
    if home_ortg > away_ortg and home_drtg < away_drtg and home_recent["Avg Points Scored"] > away_recent["Avg Points Scored"]:
        better_team = home_team
        advantage_explanation = f"{home_team} has the edge due to:\n- **Higher Offensive Rating** ({home_ortg} vs {away_ortg})\n- **Better Defense** ({home_drtg} vs {away_drtg})\n- **Recent Scoring Advantage** ({home_recent['Avg Points Scored']} vs {away_recent['Avg Points Scored']})"
    elif away_ortg > home_ortg and away_drtg < home_drtg and away_recent["Avg Points Scored"] > home_recent["Avg Points Scored"]:
        better_team = away_team
        advantage_explanation = f"{away_team} has the edge due to:\n- **Higher Offensive Rating** ({away_ortg} vs {home_ortg})\n- **Better Defense** ({away_drtg} vs {home_drtg})\n- **Recent Scoring Advantage** ({away_recent['Avg Points Scored']} vs {home_recent['Avg Points Scored']})"
    else:
        better_team = "Balanced matchup"
        advantage_explanation = "Both teams are evenly matched in offense and defense."

    # Convert abbreviations to full team names for output
    home_team_full = reverse_team_name_mapping.get(home_team, home_team)
    away_team_full = reverse_team_name_mapping.get(away_team, away_team)

    # Return the structured output
    return {
        "Home Team": home_team_full,
        "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace} | Recent Avg Pts Scored: {home_recent['Avg Points Scored']} | Recent Avg Pts Allowed: {home_recent['Avg Points Allowed']}",
        "Away Team": away_team_full,
        "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace} | Recent Avg Pts Scored: {away_recent['Avg Points Scored']} | Recent Avg Pts Allowed: {away_recent['Avg Points Allowed']}",
        "Better Team": better_team,
        "Advantage Explanation": advantage_explanation,
        "Prop Bets": prop_bets
    }

    return matchups

# Run the Analysis
schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

matchups = []  # Store matchup results

for game in schedule:
    home_team = game["home_team"]
    away_team = game["away_team"]

    # Ensure both teams exist in the metrics dataset before analyzing
    if home_team in team_metrics["TEAM_ABBREVIATION"].values and away_team in team_metrics["TEAM_ABBREVIATION"].values:
        matchup_result = analyze_matchup(home_team, away_team, team_metrics)
        matchups.append(matchup_result)
    else:
        print(f"Skipping matchup {home_team} vs {away_team} due to missing team metrics.")

# Print Results
for match in matchups:
    print(json.dumps(match, indent=4))

Today's Schedule: [{'home_team': 'PHI', 'away_team': 'DAL'}, {'home_team': 'CLE', 'away_team': 'BOS'}, {'home_team': 'BKN', 'away_team': 'HOU'}, {'home_team': 'TOR', 'away_team': 'NYK'}, {'home_team': 'CHI', 'away_team': 'MIA'}, {'home_team': 'LAC', 'away_team': 'LAL'}, {'home_team': 'POR', 'away_team': 'IND'}]
Error fetching last 5 games for PHI: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for DAL: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for CLE: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for BOS: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for BKN: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for HOU: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for T

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
from nba_api.stats.endpoints import teamgamelog, scoreboardv2

pd.set_option('display.max_columns', None)

# Mapping team full names to abbreviations
team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}

# Reverse mapping (Abbreviation -> Full Name)
team_abbreviation_to_name = {abbr: full_name for full_name, abbr in team_name_mapping.items()}

# Get today's schedule
def fetch_schedule():
    try:
        url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        est = pytz.timezone('US/Eastern')
        today = datetime.now(est).strftime('%Y-%m-%d')
        games_today = []

        for month_data in data['lscd']:
            for game in month_data['mscd']['g']:
                if game['gdte'] == today:
                    games_today.append({'home_team': game['h']['ta'], 'away_team': game['v']['ta']})

        return games_today
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []

# Fetch last 5 games stats for a team
def fetch_last_5_games_avg(team_abbr):
    try:
        game_log = teamgamelog.TeamGameLog(team_id=team_abbr, season="2024-25", season_type="Regular Season").get_data_frames()[0]
        last_5_games = game_log.head(5)

        # Compute averages
        off_rtg_avg = last_5_games['OFF_RATING'].mean()
        def_rtg_avg = last_5_games['DEF_RATING'].mean()
        pace_avg = last_5_games['PACE'].mean()

        return off_rtg_avg, def_rtg_avg, pace_avg
    except Exception as e:
        print(f"Error fetching last 5 games for {team_abbr}: {e}")
        return None, None, None

# Analyze matchup based on last 5 games
def analyze_matchup(home_team, away_team):
    home_ortg, home_drtg, home_pace = fetch_last_5_games_avg(home_team)
    away_ortg, away_drtg, away_pace = fetch_last_5_games_avg(away_team)

    if None in [home_ortg, home_drtg, home_pace, away_ortg, away_drtg, away_pace]:
        return None  # Skip if data is missing

    home_team_full = team_abbreviation_to_name.get(home_team, home_team)
    away_team_full = team_abbreviation_to_name.get(away_team, away_team)

    # Compute averages
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2
    avg_pace = (home_pace + away_pace) / 2

    # Determine best prop bets
    if avg_ortg >= 115 and avg_pace >= 100:
        prop_bets = "High-scoring game (Favoring Points & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97:
        prop_bets = "Slow-paced game (Favoring Unders for Points Props)"
    elif avg_ortg >= 115 and avg_drtg <= 110:
        prop_bets = "Fast-paced game (High Points Props)"
    elif avg_pace >= 100:
        prop_bets = "Fast-paced game (More Possessions Big Upside for Points, Rebounds & Assists Props)"
    elif avg_drtg < 110 and avg_pace <= 97:
        prop_bets = "Low-paced game (Likely to Hit Unders)"
    else:
        prop_bets = "Neutral Matchup"

    # Determine better team
    if home_ortg > away_ortg and home_drtg < away_drtg:
        better_team = home_team_full
        advantage_explanation = f"{home_team_full} has the edge due to:\n- **Higher Offensive Rating** ({home_ortg} vs {away_ortg})\n- **Better Defense** ({home_drtg} vs {away_drtg})"
    elif away_ortg > home_ortg and away_drtg < home_drtg:
        better_team = away_team_full
        advantage_explanation = f"{away_team_full} has the edge due to:\n- **Higher Offensive Rating** ({away_ortg} vs {home_ortg})\n- **Better Defense** ({away_drtg} vs {home_drtg})"
    else:
        better_team = "Balanced matchup"
        advantage_explanation = "Both teams are evenly matched in offense and defense."

    return {
        "Home Team": home_team_full,
        "Home Stats": f"OffRtg: {home_ortg:.1f} | DefRtg: {home_drtg:.1f} | Pace: {home_pace:.1f}",
        "Away Team": away_team_full,
        "Away Stats": f"OffRtg: {away_ortg:.1f} | DefRtg: {away_drtg:.1f} | Pace: {away_pace:.1f}",
        "Better Team": better_team,
        "Advantage Explanation": advantage_explanation,
        "Prop Bets": prop_bets
    }

# Run analysis
schedule = fetch_schedule()
matchups = []

for game in schedule:
    home_team = game["home_team"]
    away_team = game["away_team"]

    matchup_result = analyze_matchup(home_team, away_team)
    if matchup_result:
        matchups.append(matchup_result)
    else:
        print(f"Skipping matchup {home_team} vs {away_team} due to missing data.")

# Print Results
for match in matchups:
    print(json.dumps(match, indent=4))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 1.8 MB/s eta 0:00:00
Error fetching last 5 games for CHA: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for MIL: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Skipping matchup CHA vs MIL due to missing data.
Error fetching last 5 games for DET: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for CLE: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Skipping matchup DET vs CLE due to missing data.
Error fetching last 5 games for ATL: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetching last 5 games for SAS: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Skipping matchup ATL vs SAS due to missing data.
Error fetching last 5 games for BKN: TeamGameLog.__init__() got an unexpected keyword argument 'season_type'
Error fetc

In [ ]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics
from scipy.stats import zscore

pd.set_option('display.max_columns', None)

# Team name mappings
team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}
team_abbreviation_to_name = {abbr: full_name for full_name, abbr in team_name_mapping.items()}

def fetch_schedule():
    """Fetch today's NBA schedule."""
    url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
    response = requests.get(url)
    data = response.json()

    est = pytz.timezone('US/Eastern')
    today = datetime.now(est).strftime('%Y-%m-%d')

    return [
        {'home_team': game['h']['ta'], 'away_team': game['v']['ta']}
        for month in data['lscd']
        for game in month['mscd']['g']
        if game['gdte'] == today
    ]

def fetch_team_metrics():
    """Fetch current team efficiency metrics."""
    team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
    team_metrics = team_metrics_data.get_data_frames()[0]
    team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
    team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)

    return team_metrics

def calculate_thresholds(team_metrics):
    """Calculate percentile, Z-score, and absolute thresholds for ORtg, DRtg, and Pace."""
    thresholds = {}

    for stat in ["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]:
        values = team_metrics[stat]

        # Percentile-based (Quartiles)
        high_percentile = values.quantile(0.75)
        low_percentile = values.quantile(0.25)

        # Z-score method (above/below 1 std deviation)
        z_scores = zscore(values)
        high_zscore = values[z_scores > 1].mean() if any(z_scores > 1) else high_percentile
        low_zscore = values[z_scores < -1].mean() if any(z_scores < -1) else low_percentile

        # Absolute thresholds (based on historical NBA trends)
        absolute_high = {"E_OFF_RATING": 115, "E_DEF_RATING": 105, "E_PACE": 100}  # Example values
        absolute_low = {"E_OFF_RATING": 108, "E_DEF_RATING": 112, "E_PACE": 96}

        thresholds[stat] = {
            "high_percentile": high_percentile, "low_percentile": low_percentile,
            "high_zscore": high_zscore, "low_zscore": low_zscore,
            "absolute_high": absolute_high[stat], "absolute_low": absolute_low[stat]
        }

    return thresholds

def analyze_matchup(home_team, away_team, team_metrics, thresholds):
    """Analyze a matchup and determine the better team and prop bet recommendations."""
    home_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team].iloc[0]
    away_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team].iloc[0]

    home_team_full = team_abbreviation_to_name.get(home_team, home_team)
    away_team_full = team_abbreviation_to_name.get(away_team, away_team)

    # Extract team metrics
    home_ortg, home_drtg, home_pace = home_stats["E_OFF_RATING"], home_stats["E_DEF_RATING"], home_stats["E_PACE"]
    away_ortg, away_drtg, away_pace = away_stats["E_OFF_RATING"], away_stats["E_DEF_RATING"], away_stats["E_PACE"]

    # Determine better team
    better_team = home_team_full if (home_ortg - home_drtg) > (away_ortg - away_drtg) else away_team_full
    advantage_explanation = f"{better_team} has a stronger net efficiency rating (ORtg - DRtg)."

    # Average matchup stats
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2
    avg_pace = (home_pace + away_pace) / 2

    # Apply threshold methods
    ortg_threshold = thresholds["E_OFF_RATING"]
    drtg_threshold = thresholds["E_DEF_RATING"]
    pace_threshold = thresholds["E_PACE"]

    if avg_ortg >= ortg_threshold["high_percentile"] and avg_pace >= pace_threshold["high_percentile"]:
        prop_bets = "High-scoring game (Favoring Points & Assists Props)"
    elif avg_drtg < drtg_threshold["low_zscore"] and avg_pace <= pace_threshold["low_zscore"]:
        prop_bets = "Slow-paced game (Favoring Unders for Points Props)"
    elif avg_ortg >= ortg_threshold["absolute_high"] and avg_drtg <= drtg_threshold["absolute_low"]:
        prop_bets = "Fast-paced game (High Points Props)"
    elif avg_pace >= pace_threshold["absolute_high"]:
        prop_bets = "Fast-paced game (Big Upside for Points, Rebounds & Assists Props)"
    else:
        prop_bets = "Neutral Matchup"

    return {
        "Home Team": home_team_full,
        "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace}",
        "Away Team": away_team_full,
        "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace}",
        "Better Team": better_team,
        "Advantage Explanation": advantage_explanation,
        "Prop Bets": prop_bets
    }

# Run the Analysis
schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

if not team_metrics.empty:
    thresholds = calculate_thresholds(team_metrics)
    matchups = [analyze_matchup(game["home_team"], game["away_team"], team_metrics, thresholds) for game in schedule]
    print(json.dumps(matchups, indent=4))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 3.6 MB/s eta 0:00:00
[
    {
        "Home Team": "Charlotte Hornets",
        "Home Stats": "OffRtg: 105.4 | DefRtg: 111.1 | Pace: 100.6",
        "Away Team": "Milwaukee Bucks",
        "Away Stats": "OffRtg: 112.1 | DefRtg: 110.6 | Pace: 101.7",
        "Better Team": "Milwaukee Bucks",
        "Advantage Explanation": "Milwaukee Bucks has a stronger net efficiency rating (ORtg - DRtg).",
        "Prop Bets": "Fast-paced game (Big Upside for Points, Rebounds & Assists Props)"
    },
    {
        "Home Team": "Detroit Pistons",
        "Home Stats": "OffRtg: 109.6 | DefRtg: 111.0 | Pace: 101.8",
        "Away Team": "Cleveland Cavaliers",
        "Away Stats": "OffRtg: 120.1 | DefRtg: 108.7 | Pace: 102.3",
        "Better Team": "Cleveland Cavaliers",
        "Advantage Explanation": "Cleveland Cavaliers has a stronger net efficiency rating (ORtg - DRtg).",
        "Prop Bets": "High-scoring game (Favoring Points & Assists P

In [1]:
!pip install nba_api
import json
import pandas as pd
import requests
from datetime import datetime
import pytz
from nba_api.stats.endpoints import teamestimatedmetrics
from scipy.stats import zscore

pd.set_option('display.max_columns', None)

# Team name mappings
team_name_mapping = {
    "Atlanta Hawks": "ATL", "Boston Celtics": "BOS", "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA", "Chicago Bulls": "CHI", "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL", "Denver Nuggets": "DEN", "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW", "Houston Rockets": "HOU", "Indiana Pacers": "IND",
    "LA Clippers": "LAC", "Los Angeles Lakers": "LAL", "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA", "Milwaukee Bucks": "MIL", "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL", "Philadelphia 76ers": "PHI", "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR", "Sacramento Kings": "SAC", "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR", "Utah Jazz": "UTA", "Washington Wizards": "WAS",
}
team_abbreviation_to_name = {abbr: full_name for full_name, abbr in team_name_mapping.items()}

def fetch_schedule():
    """Fetch today's NBA schedule."""
    url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2024/league/00_full_schedule.json"
    response = requests.get(url)
    data = response.json()

    est = pytz.timezone('US/Eastern')
    today = datetime.now(est).strftime('%Y-%m-%d')

    return [
        {'home_team': game['h']['ta'], 'away_team': game['v']['ta']}
        for month in data['lscd']
        for game in month['mscd']['g']
        if game['gdte'] == today
    ]

def fetch_team_metrics():
    """Fetch current team efficiency metrics."""
    team_metrics_data = teamestimatedmetrics.TeamEstimatedMetrics(season="2024-25", season_type="Regular Season")
    team_metrics = team_metrics_data.get_data_frames()[0]
    team_metrics = team_metrics[["TEAM_NAME", "E_OFF_RATING", "E_DEF_RATING", "E_PACE"]]
    team_metrics["TEAM_ABBREVIATION"] = team_metrics["TEAM_NAME"].map(team_name_mapping)

    return team_metrics

def calculate_thresholds(team_metrics):
    """Calculate thresholds for OffRtg, DefRtg, and Pace."""
    thresholds = {}

    for stat in ["E_OFF_RATING", "E_DEF_RATING", "E_PACE"]:
        values = team_metrics[stat]

        # Percentile-based (Quartiles)
        high_percentile = values.quantile(0.75)
        low_percentile = values.quantile(0.25)

        # Z-score method (above/below 1 std deviation)
        z_scores = zscore(values)
        high_zscore = values[z_scores > 1].mean() if any(z_scores > 1) else high_percentile
        low_zscore = values[z_scores < -1].mean() if any(z_scores < -1) else low_percentile

        # Absolute thresholds
        absolute_high = {"E_OFF_RATING": 115, "E_DEF_RATING": 105, "E_PACE": 100}
        absolute_low = {"E_OFF_RATING": 108, "E_DEF_RATING": 112, "E_PACE": 96}

        thresholds[stat] = {
            "high_percentile": high_percentile, "low_percentile": low_percentile,
            "high_zscore": high_zscore, "low_zscore": low_zscore,
            "absolute_high": absolute_high[stat], "absolute_low": absolute_low[stat]
        }

    return thresholds

def predict_winner_and_props(home_team, away_team, team_metrics, thresholds, hca=2.5):
    """Predict game winner using ORTG, DRTG, and Pace, and determine best prop bets."""
    home_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == home_team].iloc[0]
    away_stats = team_metrics[team_metrics["TEAM_ABBREVIATION"] == away_team].iloc[0]

    home_team_full = team_abbreviation_to_name.get(home_team, home_team)
    away_team_full = team_abbreviation_to_name.get(away_team, away_team)

    # Extract team metrics
    home_ortg, home_drtg, home_pace = home_stats["E_OFF_RATING"], home_stats["E_DEF_RATING"], home_stats["E_PACE"]
    away_ortg, away_drtg, away_pace = away_stats["E_OFF_RATING"], away_stats["E_DEF_RATING"], away_stats["E_PACE"]

    # Calculate expected scores
    avg_pace = (home_pace + away_pace) / 2
    home_expected_score = ((home_ortg + away_drtg) / 2) * (avg_pace / 100)
    away_expected_score = ((away_ortg + home_drtg) / 2) * (avg_pace / 100)

    # Adjust for home-court advantage
    home_expected_score += hca

    # Calculate expected margin
    expected_margin = home_expected_score - away_expected_score

    # Predict winner
    winner = home_team_full if expected_margin > 0 else away_team_full

    # Determine prop bet recommendations
    avg_ortg = (home_ortg + away_ortg) / 2
    avg_drtg = (home_drtg + away_drtg) / 2

    ortg_threshold = thresholds["E_OFF_RATING"]
    drtg_threshold = thresholds["E_DEF_RATING"]
    pace_threshold = thresholds["E_PACE"]

    if avg_ortg >= ortg_threshold["high_percentile"] and avg_pace >= pace_threshold["high_percentile"]:
        prop_bets = "High-scoring game (Favoring Points & Assists Props)"
    elif avg_drtg < drtg_threshold["low_zscore"] and avg_pace <= pace_threshold["low_zscore"]:
        prop_bets = "Slow-paced game (Favoring Unders for Points Props)"
    elif avg_ortg >= ortg_threshold["absolute_high"] and avg_drtg <= drtg_threshold["absolute_low"]:
        prop_bets = "Fast-paced game (High Points Props)"
    elif avg_pace >= pace_threshold["absolute_high"]:
        prop_bets = "Fast-paced game (Big Upside for Points, Rebounds & Assists Props)"
    else:
        prop_bets = "Neutral Matchup"

    return {
        "Home Team": home_team_full,
        "Home Stats": f"OffRtg: {home_ortg} | DefRtg: {home_drtg} | Pace: {home_pace}",
        "Away Team": away_team_full,
        "Away Stats": f"OffRtg: {away_ortg} | DefRtg: {away_drtg} | Pace: {away_pace}",
        "Predicted Winner": winner,
        "Expected Margin": round(abs(expected_margin), 1),
        "Prop Bets": prop_bets
    }

# Run the Analysis
schedule = fetch_schedule()
team_metrics = fetch_team_metrics()

if not team_metrics.empty:
    thresholds = calculate_thresholds(team_metrics)
    matchups = [predict_winner_and_props(game["home_team"], game["away_team"], team_metrics, thresholds) for game in schedule]
    print(json.dumps(matchups, indent=4))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 5.4 MB/s eta 0:00:00
[
    {
        "Home Team": "Detroit Pistons",
        "Home Stats": "OffRtg: 111.7 | DefRtg: 109.6 | Pace: 102.6",
        "Away Team": "Sacramento Kings",
        "Away Stats": "OffRtg: 113.5 | DefRtg: 113.3 | Pace: 100.9",
        "Predicted Winner": "Detroit Pistons",
        "Expected Margin": 3.5,
        "Prop Bets": "Fast-paced game (Big Upside for Points, Rebounds & Assists Props)"
    },
    {
        "Home Team": "Miami Heat",
        "Home Stats": "OffRtg: 110.3 | DefRtg: 110.6 | Pace: 98.2",
        "Away Team": "Philadelphia 76ers",
        "Away Stats": "OffRtg: 109.2 | DefRtg: 115.3 | Pace: 99.7",
        "Predicted Winner": "Miami Heat",
        "Expected Margin": 5.4,
        "Prop Bets": "Neutral Matchup"
    }
]
